In [1]:
print('b')

b


### This is the file to chanage the original merged tripelt.txt to 2-hop graph
- use BFS

In [2]:
import os
from collections import defaultdict, deque
from tqdm.notebook import tqdm
import random

In [3]:
def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.readlines()

- Final version that can get all the 2-hop neighbors 
- Not merged the grpah yet
- the version of directly output the relation [......]

In [4]:
def process_data(data_lines):
    connections = defaultdict(dict)
    node_labels = {}

    for line in tqdm(data_lines, desc="Processing data lines"):
        parts = line.strip().split()
        src, dest, edge_info, label = int(parts[0]), int(parts[1]), parts[2], parts[3]

        connections[src][dest] = (edge_info, label)
        if src not in node_labels:
            node_labels[src] = label

    return connections, node_labels


def bfs_2hop(connections, start_node):
    visited = set()
    queue = deque([(start_node, 0)])  # Queue will store a tuple of (node, hops)

    while queue:
        current_node, hops = queue.popleft()
        visited.add(current_node)

        if hops < 2:  # Only proceed if we are within 2 hops
            # Check for neighbors where the current node is the source
            for neigh in connections[current_node]:
                if neigh not in visited:
                    queue.append((neigh, hops + 1))

            # Check for neighbors where the current node is the destination
            for src, dests in connections.items():
                if current_node in dests and src not in visited:
                    queue.append((src, hops + 1))

    return visited - {start_node}  # Exclude the start_node itself

- the version of getting the data with dictionary:
- source node: [......]

In [5]:
def process_data(data_lines):
    connections = defaultdict(dict)
    node_labels = {}

    for line in tqdm(data_lines, desc="Processing data lines"):
        parts = line.strip().split()
        src, dest, edge_info, label = int(parts[0]), int(parts[1]), parts[2], parts[3]

        connections[src][dest] = (edge_info, label)
        if src not in node_labels:
            node_labels[src] = label

    return connections, node_labels


def bfs_2hop(connections, start_node):
    visited = set()
    queue = deque([(start_node, 0)])  # Queue will store a tuple of (node, hops)

    while queue:
        current_node, hops = queue.popleft()
        visited.add(current_node)

        if hops < 2:  # Only proceed if we are within 2 hops
            # Check for neighbors where the current node is the source
            for neigh in connections[current_node]:
                if neigh not in visited:
                    queue.append((neigh, hops + 1))

            # Check for neighbors where the current node is the destination
            for src, dests in connections.items():
                if current_node in dests and src not in visited:
                    queue.append((src, hops + 1))

    return visited - {start_node}  # Exclude the start_node itselfx

In [7]:
def extract_subgraphs(connections, node_labels, label_type):
    subgraphs = {}

    for node, label in tqdm(node_labels.items(), desc=f"Extracting {label_type} subgraphs"):
        if label == label_type:
            neighbors_2hop = bfs_2hop(connections, node)
            subgraph = []

            for neigh in neighbors_2hop.union({node}):  # 包括节点本身
                if neigh in connections:  # 检查邻居是否有任何连接
                    for dest, (edge_info, edge_label) in connections[neigh].items():
                        if dest in neighbors_2hop or dest == node:  # 包括2-hop内的边或返回源节点
                            subgraph_line = f"{neigh} {dest} {edge_info} {edge_label}"
                            subgraph.append(subgraph_line)

            if subgraph:  # 仅当子图不为空时添加
                subgraphs[node] = subgraph  # 使用源节点作为键

    return subgraphs

# subgraphs_attack = extract_subgraphs(connections, node_labels, 'attack')
# subgraphs_benign = extract_subgraphs(connections, node_labels, 'benign')


# print(subgraphs_benign, len(subgraphs_benign), '\n\n')
# print(subgraphs_attack, len(subgraphs_attack))

In [8]:
def merge_overlapping_subgraphs(subgraphs_dict):
    all_edges = set()
    for edges in subgraphs_dict.values():
        all_edges.update(edges)

    merged_subgraphs = []
    graph_map = {}  # A mapping of node to graph index in merged_subgraphs

    for edge in tqdm(all_edges, desc="Merging subgraphs"):
        src, dest, *_ = edge.split()

        src_graph = graph_map.get(src)
        dest_graph = graph_map.get(dest)

        if src_graph is not None and dest_graph is not None:
            if src_graph != dest_graph:
                # Merge the two graphs
                merged_subgraphs[src_graph].update(merged_subgraphs[dest_graph])
                merged_subgraphs[dest_graph] = merged_subgraphs[src_graph]
                # Update the graph_map for all nodes in the merged graph
                for node in merged_subgraphs[src_graph]:
                    graph_map[node] = src_graph
            merged_subgraphs[src_graph].add(edge)
        elif src_graph is not None:
            # Add to src graph
            merged_subgraphs[src_graph].add(edge)
            graph_map[dest] = src_graph
        elif dest_graph is not None:
            # Add to dest graph
            merged_subgraphs[dest_graph].add(edge)
            graph_map[src] = dest_graph
        else:
            # Create a new graph
            new_index = len(merged_subgraphs)
            merged_subgraphs.append({edge})
            graph_map[src] = graph_map[dest] = new_index

    # Remove duplicate graphs
    unique_subgraphs = []
    for subgraph in merged_subgraphs:
        if subgraph not in unique_subgraphs:
            unique_subgraphs.append(subgraph)

    return unique_subgraphs

In [89]:
merged_graphs = merge_overlapping_subgraphs(subgraphs_benign)
merged_graphs

Merging subgraphs:   0%|          | 0/7 [00:00<?, ?it/s]

[{'10 1000 2 benign',
  '1000 500 1 benign',
  '24027 8575 14 benign',
  '24027 8787 10 benign',
  '24462 8575 2 benign',
  '8575 10 2 benign'},
 {'1 3 5 benign'}]

In [92]:
merged_graphs = merge_overlapping_subgraphs(subgraphs_attack)
merged_graphs

Merging subgraphs:   0%|          | 0/2 [00:00<?, ?it/s]

[{'16301 10385 6 attack'}, {'16301 2 100 benign'}]

In [9]:
def merge_overlapping_subgraphs(subgraphs_dict):
    all_edges = set()
    for edges in subgraphs_dict.values():
        all_edges.update(edges)

    merged_subgraphs = []
    graph_map = {}  # A mapping of node to graph index in merged_subgraphs

    for edge in tqdm(all_edges, desc="Merging subgraphs"):
        src, dest, *_ = edge.split()

        src_graph = graph_map.get(src)
        dest_graph = graph_map.get(dest)

        if src_graph is not None:
            # Add to src graph
            merged_subgraphs[src_graph].add(edge)
            graph_map[dest] = src_graph
        elif dest_graph is not None:
            # Add to dest graph
            merged_subgraphs[dest_graph].add(edge)
            graph_map[src] = dest_graph
        else:
            # Create a new graph
            new_index = len(merged_subgraphs)
            merged_subgraphs.append({edge})
            graph_map[src] = graph_map[dest] = new_index

    # Remove duplicate graphs
    unique_subgraphs = []
    for subgraph in merged_subgraphs:
        if subgraph not in unique_subgraphs:
            unique_subgraphs.append(subgraph)

    return unique_subgraphs


In [96]:
merged_graphs = merge_overlapping_subgraphs(subgraphs_attack)
print(len(merged_graphs))
merged_graphs


Merging subgraphs:   0%|          | 0/2 [00:00<?, ?it/s]

1


[{'16301 10385 6 attack', '16301 2 100 benign'}]

In [97]:
merged_graphs = merge_overlapping_subgraphs(subgraphs_benign)
print(len(merged_graphs))
merged_graphs

Merging subgraphs:   0%|          | 0/7 [00:00<?, ?it/s]

2


[{'10 1000 2 benign',
  '1000 500 1 benign',
  '24027 8575 14 benign',
  '24027 8787 10 benign',
  '24462 8575 2 benign',
  '8575 10 2 benign'},
 {'1 3 5 benign'}]

In [10]:
def save_merged_subgraphs_to_files(merged_subgraphs, output_folder, label_type):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, subgraph in enumerate(tqdm(merged_subgraphs, desc=f"Saving {label_type} merged subgraphs")):
        file_name = os.path.join(output_folder, f"{label_type}_merged_subgraph_{i}.txt")
        with open(file_name, 'w') as file:
            for edge in subgraph:
                file.write(f"{edge}\n")  # 直接写入边的字符串
        print(f"{file_name} saved")


In [11]:
output_folder = '../data/source_data/output_graphs(200)'

save_merged_subgraphs_to_files(merged_graphs, output_folder, 'benign')

NameError: name 'merged_graphs' is not defined

- for the final dataset

In [14]:
input_file_path = './data/source_data/merged_all.txt'  # Update with your actual file path
data_lines = read_data_from_file(input_file_path)
connections, labels, label_to_nodes = process_data(data_lines)

output_folder = './data/source_data/output_graphs(200)'  # Update with your actual output folder path

subgraphs_benign = extract_subgraphs(connections, label_to_nodes, 'benign', 200)
subgraphs_attack = extract_subgraphs(connections, label_to_nodes, 'attack', 200)

merged_subgraphs_benign = merge_overlapping_subgraphs(subgraphs_benign)
merged_subgraphs_attack = merge_overlapping_subgraphs(subgraphs_attack)

save_merged_subgraphs_to_files(merged_subgraphs_benign, output_folder, 'benign')
save_merged_subgraphs_to_files(merged_subgraphs_attack, output_folder, 'attack')

Processing data lines:   0%|          | 0/51479769 [00:00<?, ?it/s]

ValueError: not enough values to unpack (expected 3, got 2)

## the complete code

- 先建立大圖
    - networkx 的neighbor function 試試看
    
- 讀全部tuple進來
    - 建立set of source destination node sets
    - node 2 entity dictionary for neighbor
    - node 2 event list
    - attack 優先算
    - networkx 最後一步

In [2]:
import os
from collections import defaultdict, deque
from tqdm.notebook import tqdm
import random

In [4]:
def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        print(f"read file from: {file_path}")
        return file.readlines()

def process_data(data_lines):
    connections = defaultdict(dict)
    node_labels = {}

    for line in tqdm(data_lines, desc="Processing data lines"):
        parts = line.strip().split()
        src, dest, edge_info, label = int(parts[0]), int(parts[1]), parts[2], parts[3]

        connections[src][dest] = (edge_info, label)
        if src not in node_labels:
            node_labels[src] = label

    return connections, node_labels


def bfs_2hop(connections, start_node):
    visited = set()
    queue = deque([(start_node, 0)])  # Queue will store a tuple of (node, hops)

    while queue:
        current_node, hops = queue.popleft()
        visited.add(current_node)

        if hops < 2:  # Only proceed if we are within 2 hops
            # Check for neighbors where the current node is the source
            for neigh in connections[current_node]:
                if neigh not in visited:
                    queue.append((neigh, hops + 1))

            # Check for neighbors where the current node is the destination
            for src, dests in connections.items():
                if current_node in dests and src not in visited:
                    queue.append((src, hops + 1))

    return visited - {start_node}  # Exclude the start_node itself


def extract_subgraphs(connections, node_labels, label_type):
    subgraphs = {}

#     for node, label in tqdm(node_labels.items(), desc=f"Extracting {label_type} subgraphs"):
    for node, label in node_labels.items():
        if label == label_type:
            neighbors_2hop = bfs_2hop(connections, node)
            subgraph = []

            for neigh in neighbors_2hop.union({node}):  # 包括节点本身
                if neigh in connections:  # 检查邻居是否有任何连接
                    for dest, (edge_info, edge_label) in connections[neigh].items():
                        if dest in neighbors_2hop or dest == node:  # 包括2-hop内的边或返回源节点
                            subgraph_line = f"{neigh} {dest} {edge_info} {edge_label}"
                            subgraph.append(subgraph_line)

            if subgraph:  # 仅当子图不为空时添加
                subgraphs[node] = subgraph  # 使用源节点作为键

    return subgraphs


def merge_overlapping_subgraphs(subgraphs_dict):
    all_edges = set()
    for edges in subgraphs_dict.values():
        all_edges.update(edges)

    merged_subgraphs = []
    graph_map = {}  # A mapping of node to graph index in merged_subgraphs

    for edge in tqdm(all_edges, desc="Merging subgraphs"):
        src, dest, *_ = edge.split()

        src_graph = graph_map.get(src)
        dest_graph = graph_map.get(dest)

        if src_graph is not None and dest_graph is not None:
            if src_graph != dest_graph:
                # Merge the two graphs
                merged_subgraphs[src_graph].update(merged_subgraphs[dest_graph])
                merged_subgraphs[dest_graph] = merged_subgraphs[src_graph]
                # Update the graph_map for all nodes in the merged graph
                for node in merged_subgraphs[src_graph]:
                    graph_map[node] = src_graph
            merged_subgraphs[src_graph].add(edge)
        elif src_graph is not None:
            # Add to src graph
            merged_subgraphs[src_graph].add(edge)
            graph_map[dest] = src_graph
        elif dest_graph is not None:
            # Add to dest graph
            merged_subgraphs[dest_graph].add(edge)
            graph_map[src] = dest_graph
        else:
            # Create a new graph
            new_index = len(merged_subgraphs)
            merged_subgraphs.append({edge})
            graph_map[src] = graph_map[dest] = new_index

    # Remove duplicate graphs
    unique_subgraphs = []
    for subgraph in merged_subgraphs:
        if subgraph not in unique_subgraphs:
            unique_subgraphs.append(subgraph)

    return unique_subgraphs


def save_merged_subgraphs_to_files(merged_subgraphs, output_folder, label_type):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, subgraph in enumerate(tqdm(merged_subgraphs, desc=f"Saving {label_type} merged subgraphs")):
        file_name = os.path.join(output_folder, f"{label_type}_merged_subgraph_{i}.txt")
        with open(file_name, 'w') as file:
            for edge in subgraph:
                file.write(f"{edge}\n")
        print(f"{file_name} saved")

In [ ]:
def select_random_sources(node_labels, label_type, count):
    nodes = [node for node, label in node_labels.items() if label == label_type]
    return random.sample(nodes, min(count, len(nodes)))

def main_process(connections, node_labels, iteration, base_output_folder):
    # choose random source nodes
    benign_sources = select_random_sources(node_labels, 'benign', 200)
    attack_sources = select_random_sources(node_labels, 'attack', 200)

    # get subgraph
    subgraphs_benign = {}
    for src in benign_sources:
        temp_subgraphs = extract_subgraphs(connections, {src: node_labels[src]}, 'benign')
        subgraphs_benign.update(temp_subgraphs)

    subgraphs_attack = {}
    for src in attack_sources:
        temp_subgraphs = extract_subgraphs(connections, {src: node_labels[src]}, 'attack')
        subgraphs_attack.update(temp_subgraphs)

    # merge the overlapping subgraph
    merged_subgraphs_benign = merge_overlapping_subgraphs(subgraphs_benign)
    merged_subgraphs_attack = merge_overlapping_subgraphs(subgraphs_attack)

    # save into the output folder with different name
    output_folder = f"{base_output_folder}-{iteration}"
    save_merged_subgraphs_to_files(merged_subgraphs_benign, output_folder, 'benign')
    save_merged_subgraphs_to_files(merged_subgraphs_attack, output_folder, 'attack')

    
input_file_path = '/workdir/home/bai/Euni_HO_modified/data/source_data/merged_all_sorted.txt'
data_lines = read_data_from_file(input_file_path)
connections, node_labels = process_data(data_lines)

base_output_folder = '/workdir/home/bai/Euni_HO_modified/data/source_data/output_graphs(200)/output_graphs(200)'
for i in range(10):
    main_process(connections, node_labels, i, base_output_folder)

read file from: /workdir/home/bai/Euni_HO_modified/data/source_data/merged_all_sorted.txt


Processing data lines:   0%|          | 0/51479769 [00:01<?, ?it/s]

Merging subgraphs:   0%|          | 0/421185 [00:00<?, ?it/s]

Merging subgraphs:   0%|          | 0/426225 [00:00<?, ?it/s]

In [ ]:
input_file_path = '/workdir/home/bai/Euni_HO_modified/data/source_data/merged_all.txt'  # Update with your actual file path
data_lines = read_data_from_file(input_file_path)
connections, labels, label_to_nodes = process_data(data_lines)

output_folder = '/workdir/home/bai/Euni_HO_modified/data/source_data/output_graphs(200)'  # Update with your actual

In [80]:
import os
from collections import defaultdict, deque
from tqdm.notebook import tqdm

def process_data(data_lines):
    connections = defaultdict(dict)
    node_labels = {}

    for line in tqdm(data_lines, desc="Processing data lines"):
        parts = line.strip().split()
        src, dest, edge_info, label = int(parts[0]), int(parts[1]), parts[2], parts[3]

        connections[src][dest] = (edge_info, label)
        if src not in node_labels:
            node_labels[src] = label

    return connections, node_labels

# ================== not considering the backward direction ================== 
# def bfs_2hop(connections, start_node):
#     visited = set()
#     queue = deque([(start_node, 0)])  # Queue will store a tuple of (node, hops)

#     while queue:
#         current_node, hops = queue.popleft()
#         visited.add(current_node)

#         if hops < 2:  # Only proceed if we are within 2 hops
#             for neigh in connections[current_node]:
#                 if neigh not in visited:
#                     queue.append((neigh, hops + 1))

#     return visited - {start_node}  # Exclude the start_node itself

def bfs_2hop(connections, start_node):
    visited = set()
    queue = deque([(start_node, 0)])  # Queue will store a tuple of (node, hops)

    while queue:
        current_node, hops = queue.popleft()
        visited.add(current_node)

        if hops < 2:  # Only proceed if we are within 2 hops
            # Check for neighbors where the current node is the source
            for neigh in connections[current_node]:
                if neigh not in visited:
                    queue.append((neigh, hops + 1))

            # Check for neighbors where the current node is the destination
            for src, dests in connections.items():
                if current_node in dests and src not in visited:
                    queue.append((src, hops + 1))

    return visited - {start_node}  # Exclude the start_node itself




def extract_subgraphs(connections, node_labels, label_type):
    subgraphs = []

    for node, label in tqdm(node_labels.items(), desc=f"Extracting {label_type} subgraphs"):
        if label == label_type:
#             neighbors_2hop = bfs_2hop(connections, node, set())  # 不再使用 visited_global
            neighbors_2hop = bfs_2hop(connections, node)  # 不再使用 visited_global
            subgraph = []

            for neigh in neighbors_2hop.union({node}):  # 包括节点本身
                if neigh in connections:  # 检查邻居是否有任何连接
                    for dest, (edge_info, edge_label) in connections[neigh].items():
                        if dest in neighbors_2hop or dest == node:  # 包括2-hop内的边或返回源节点
                            subgraph_line = f"{neigh} {dest} {edge_info} {edge_label}"
                            subgraph.append(subgraph_line)

            if subgraph:  # 仅当子图不为空时添加
                subgraphs.append(subgraph)

    return subgraphs


data_lines = [
    "24027 8575 14 benign",
    "24027 8787 10 benign",
    "24462 8575 2 benign",
    "24462 8575 2 benign",
    "8575 10 2 benign",
    "10 1000 2 benign",
    
    # the order would affect the output -> first see first output and into the visited nodes
#     "16301 2 100 benign", 
    "16301 10385 6 attack",
    "16301 2 100 benign",
    "1 3 5 benign",
    "1000 500 1 benign"
]

connections, node_labels = process_data(data_lines)
subgraphs_attack = extract_subgraphs(connections, node_labels, 'attack')
subgraphs_benign = extract_subgraphs(connections, node_labels, 'benign')


print(subgraphs_benign, len(subgraphs_benign), '\n\n')
print(subgraphs_attack, len(subgraphs_attack))


Processing data lines:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting attack subgraphs:   0%|          | 0/7 [00:00<?, ?it/s]

Extracting benign subgraphs:   0%|          | 0/7 [00:00<?, ?it/s]

[['24027 8575 14 benign', '24027 8787 10 benign', '24462 8575 2 benign', '8575 10 2 benign'], ['24027 8575 14 benign', '24462 8575 2 benign', '8575 10 2 benign'], ['10 1000 2 benign', '24027 8575 14 benign', '24027 8787 10 benign', '24462 8575 2 benign', '8575 10 2 benign'], ['1000 500 1 benign', '10 1000 2 benign', '24027 8575 14 benign', '24462 8575 2 benign', '8575 10 2 benign'], ['1 3 5 benign'], ['1000 500 1 benign', '10 1000 2 benign', '8575 10 2 benign']] 6 


[['16301 10385 6 attack', '16301 2 100 benign']] 1


- the complete code

In [4]:
import os
from collections import defaultdict, deque
from tqdm.notebook import tqdm

def process_data(data_lines):
    edges = [tuple(map(int, line.split()[:3])) for line in tqdm(data_lines, desc="Processing data lines")]
    connections = defaultdict(set)
    labels = {}
    label_to_nodes = {'benign': set(), 'attack': set()}

    for src, dest, label in edges:
        connections[src].add(dest)
        connections[dest].add(src)
        labels[src] = label
        labels[dest] = label
        if label == 0:
            label_to_nodes['benign'].add(src)
            label_to_nodes['benign'].add(dest)
        elif label == 1:
            label_to_nodes['attack'].add(src)
            label_to_nodes['attack'].add(dest)

    return connections, labels, label_to_nodes

def bfs_2hop(connections, start_node):
    visited = set()
    queue = deque([start_node])
    hops = {start_node: 0}

    while queue:
        node = queue.popleft()
#         for neigh in tqdm(connections[node], desc=f"BFS from node {start_node}"):
        for neigh in connections[node]:
            if neigh not in hops:
                hops[neigh] = hops[node] + 1
                if hops[neigh] > 2:
                    continue
                queue.append(neigh)
                visited.add(neigh)
    return visited

def extract_subgraphs(connections, label_to_nodes, label_type, count=200):
    selected_nodes = list(label_to_nodes[label_type])[:count]
    subgraphs = []

    for node in tqdm(selected_nodes, desc=f"Extracting {label_type} subgraphs"):
        neighbors = bfs_2hop(connections, node)

        subgraph = []
        for neigh in neighbors:
            for connected_node in connections[neigh]:
                if connected_node in neighbors:
                    subgraph.append(f"{neigh} {connected_node} {labels[neigh]}")

        subgraphs.append(subgraph)

    return subgraphs

def merge_overlapping_subgraphs(subgraphs):
    merged_subgraphs = []
    visited_nodes = set()

    for subgraph in tqdm(subgraphs, desc="Merging subgraphs"):
        current_nodes = set([edge.split()[0] for edge in subgraph] + [edge.split()[1] for edge in subgraph])
        if not visited_nodes.isdisjoint(current_nodes):
            overlap_graph = next((g for g in merged_subgraphs if not set(g.keys()).isdisjoint(current_nodes)), None)
            if overlap_graph:
                for src, dest, _ in [edge.split() for edge in subgraph]:
                    overlap_graph[src].add(dest)
            else:
                merged_subgraph = defaultdict(set)
                for src, dest, _ in [edge.split() for edge in subgraph]:
                    merged_subgraph[src].add(dest)
                merged_subgraphs.append(merged_subgraph)
        else:
            new_subgraph = defaultdict(set)
            for src, dest, _ in [edge.split() for edge in subgraph]:
                new_subgraph[src].add(dest)
            merged_subgraphs.append(new_subgraph)

        visited_nodes.update(current_nodes)

    return merged_subgraphs

def save_merged_subgraphs_to_files(merged_subgraphs, output_folder, label_type):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, subgraph in enumerate(tqdm(merged_subgraphs, desc=f"Saving {label_type} merged subgraphs")):
        file_name = os.path.join(output_folder, f"{label_type}_merged_subgraph_{i}.txt")
        with open(file_name, 'w') as file:
            for src, edges in subgraph.items():
                for dest in edges:
                    file.write(f"{src} {dest}\n")
        print(f"{file_name} saved")

def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.readlines()

input_file_path = '/workdir/home/bai/Euni_HO_modified/data/source_data/merged_all.txt'  # Update with your actual file path
data_lines = read_data_from_file(input_file_path)
connections, labels, label_to_nodes = process_data(data_lines)

output_folder = '/workdir/home/bai/Euni_HO_modified/data/source_data/output_graphs(200)'  # Update with your actual output folder path

subgraphs_benign = extract_subgraphs(connections, label_to_nodes, 'benign', 200)
subgraphs_attack = extract_subgraphs(connections, label_to_nodes, 'attack', 200)

merged_subgraphs_benign = merge_overlapping_subgraphs(subgraphs_benign)
merged_subgraphs_attack = merge_overlapping_subgraphs(subgraphs_attack)

save_merged_subgraphs_to_files(merged_subgraphs_benign, output_folder, 'benign')
save_merged_subgraphs_to_files(merged_subgraphs_attack, output_folder, 'attack')

Processing data lines:   0%|          | 0/51479769 [00:00<?, ?it/s]

Extracting benign subgraphs:   0%|          | 0/200 [00:00<?, ?it/s]

Extracting attack subgraphs:   0%|          | 0/200 [00:00<?, ?it/s]

Merging subgraphs:   0%|          | 0/200 [00:02<?, ?it/s]

Merging subgraphs:   0%|          | 0/200 [00:00<?, ?it/s]

Saving benign merged subgraphs:   0%|          | 0/1 [00:00<?, ?it/s]

/workdir/home/bai/Euni_HO_modified/data/source_data/output_graphs(200)/benign_merged_subgraph_0.txt saved


NameError: name 'save_merged_sub' is not defined

In [5]:
save_merged_subgraphs_to_files(merged_subgraphs_attack, output_folder, 'attack')

Saving attack merged subgraphs:   0%|          | 0/1 [00:00<?, ?it/s]

/workdir/home/bai/Euni_HO_modified/data/source_data/output_graphs(200)/attack_merged_subgraph_0.txt saved


In [ ]:
import os
from collections import defaultdict, deque
from tqdm import tqdm

def process_data(data_lines):
    edges = [tuple(map(int, line.split()[:3])) for line in data_lines]
    connections = defaultdict(set)
    labels = {}
    label_to_nodes = {'benign': set(), 'attack': set()}

    for src, dest, label in edges:
        connections[src].add(dest)
        connections[dest].add(src)
        labels[src] = label
        labels[dest] = label
        if label == 0:
            label_to_nodes['benign'].add(src)
            label_to_nodes['benign'].add(dest)
        elif label == 1:
            label_to_nodes['attack'].add(src)
            label_to_nodes['attack'].add(dest)

    return connections, labels, label_to_nodes

def bfs_2hop(connections, start_node):
    visited = set()
    queue = deque([start_node])
    hops = {start_node: 0}
    while queue:
        node = queue.popleft()
        for neigh in connections[node]:
            if neigh not in hops:
                hops[neigh] = hops[node] + 1
                if hops[neigh] > 2:
                    continue
                queue.append(neigh)
                visited.add(neigh)
    return visited

def extract_subgraphs(connections, label_to_nodes, label_type, count=200):
    selected_nodes = list(label_to_nodes[label_type])[:count]
    subgraphs = []

    for node in selected_nodes:
        neighbors = bfs_2hop(connections, node)

        # Create subgraph
        subgraph = []
        for neigh in neighbors:
            for connected_node in connections[neigh]:
                if connected_node in neighbors:
                    subgraph.append(f"{neigh} {connected_node} {labels[neigh]}")

        subgraphs.append(subgraph)

    return subgraphs

In [ ]:
def save_merged_subgraphs_to_files(subgraphs, output_folder, label_type):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, subgraph in enumerate(tqdm(subgraphs, desc=f"Saving {label_type} subgraphs to files")):
        file_name = os.path.join(output_folder, f"{label_type}_subgraph_{i}.txt")
        with open(file_name, 'w') as file:
            for edge in subgraph:
                file.write(edge + '\n')
        print(f"{file_name} saved")

def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.readlines()

input_file_path = '/workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/merged_all.txt'
data_lines = read_data_from_file(input_file_path)
connections, labels, label_to_nodes = process_data(data_lines)

output_folder = '/workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/output_graphs'

# Extract and save benign subgraphs

In [ ]:
def merge_overlapping_subgraphs(subgraphs):
    merged_subgraphs = []
    visited_nodes = set()

    for subgraph in subgraphs:
        current_nodes = set([edge[0] for edge in subgraph] + [edge[1] for edge in subgraph])
        if not visited_nodes.isdisjoint(current_nodes):
            # 如果当前子图与已访问节点有重叠，合并它们
            overlap_graph = next((g for g in merged_subgraphs if not set(g.keys()).isdisjoint(current_nodes)), None)
            if overlap_graph:
                # 更新已合并的子图
                for src, dest, label in subgraph:
                    overlap_graph[src].add(dest)
            else:
                # 创建新的合并子图
                merged_subgraph = defaultdict(set)
                for src, dest, label in subgraph:
                    merged_subgraph[src].add(dest)
                merged_subgraphs.append(merged_subgraph)
        else:
            # 创建新的子图
            new_subgraph = defaultdict(set)
            for src, dest, label in subgraph:
                new_subgraph[src].add(dest)
            merged_subgraphs.append(new_subgraph)

        visited_nodes.update(current_nodes)

    return merged_subgraphs

# 先使用之前的逻辑提取所有子图
subgraphs_benign = extract_subgraphs(connections, label_to_nodes, 'benign', 200)
subgraphs_attack = extract_subgraphs(connections, label_to_nodes, 'attack', 200)

# 合并重叠的子图
merged_subgraphs_benign = merge_overlapping_subgraphs(subgraphs_benign)
merged_subgraphs_attack = merge_overlapping_subgraphs(subgraphs_attack)

# 保存合并后的子图
save_merged_subgraphs_to_files(merged_subgraphs_benign, output_folder, 'benign')
save_merged_subgraphs_to_files(merged_subgraphs_attack, output_folder, 'attack')
